# Árvore de decisão C4.5 (base de dados simples)

In [1]:
import numpy as np
import pandas as pd

### Dados (base de dados gerada manualmente)

In [2]:
# dados de entrada
x1 = [0, 1, 1, 2, 2, 2]
x2 = [0, 0, 1, 1, 1, 0]
# dados de saída (rótulos: 0 ou 1)
y = np.array([0, 0, 0, 1, 1, 0])
# dados estruturados em tabela
df = pd.DataFrame(list(zip(x1, x2, [0, 0, 0, 1, 1, 0])), columns =['x1', 'x2', 'y'])
# print df
df

,x1,x2,y
0,0,0,0
1,1,0,0
2,1,1,0
3,2,1,1
4,2,1,1
5,2,0,0


### Função para split dos dados

In [3]:
def partition(a):
    return {c: (a==c).nonzero()[0] for c in np.unique(a)}

### Função para o cálculo do entropia

In [4]:
def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts = True)
    freqs = counts.astype('float')/len(s)
    for p in freqs:
        if p != 0.0:
            res -= p*np.log2(p)
    return res

### Função para o cálculo de ganho de informação (information gain)  

In [5]:
def mutual_information(y, x):
    res = entropy(y)
    #Particionamos x de acordo com os valores dos atrinutos x_i
    val, counts = np.unique(x, return_counts = True)
    freqs = counts.astype('float')/len(x)
    #Calculamos uma média ponderada da entropia
    for p, v in zip(freqs, val):
        res -= p*entropy(y[x == v])
    return res

In [6]:
def is_pure(s):
    return len(set(s)) == 1

In [7]:
def recursive_split(x, y):
    #Se não for possível realizar o split, retornar o set original
    if is_pure(y) or len(y) == 0:
        return y
    #Escolher o atributo que fornece o maior ganho de informação
    gain = np.array([mutual_information(y, x_attr) for x_attr in x.T])
    selected_attr = np.argmax(gain)
    #Se não houver ganho, nada deve ser feito e deve-se retornar o set original
    if np.all(gain < 1e-6):
        return y
    #O split é realizado utilizando o atributo selecionado
    sets = partition(x[:, selected_attr])

    res = {}
    for k, v in sets.items():
        y_subset = y.take(v, axis = 0)
        x_subset = x.take(v, axis = 0)

        res["x_%d = %d" % (selected_attr + 1, k)] = recursive_split(x_subset, y_subset)

    return res

In [8]:
X = np.array([x1,x2]).T
print(recursive_split(X, y))

{'x_1 = 0': array([0]), 'x_1 = 1': array([0, 0]), 'x_1 = 2': {'x_2 = 0': array([0]), 'x_2 = 1': array([1, 1])}}


### Árvore de decisão gerada

<img src="../Simple Dataset/C4.5 Simples.png" />